# StatsCan shapefile processing
*April 22, 2022*

This notebook takes statscan census shapefiles and processes them into various useful maps for import into Datawrapper. First, we import geopandas, pandas, and a third module to suppress some annoying warning messages.

In [16]:
import geopandas
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

Now we read in the latest StatsCan census boundary files, and convert the coordinate system to EPSG:4326, which is what Datawrapper likes.

In [17]:
tracts = (geopandas
          .read_file("https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/files-fichiers/lct_000b21a_e.zip")
          .to_crs("EPSG:4326")
          )

In [18]:
diss_areas = (geopandas
          .read_file("https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/files-fichiers/lda_000b21a_e.zip")
          .to_crs("EPSG:4326")
          )

In [19]:
subdivisions = (geopandas
          .read_file("https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/files-fichiers/lcsd000b21a_e.zip")
          .to_crs("EPSG:4326")
          )

We'll add a column here to pull the CMA code from the DGUID for later use.

In [20]:
tracts["CMA_CODE"] = tracts["DGUID"].str[9:12]
tracts

,CTUID,DGUID,CTNAME,LANDAREA,PRUID,geometry,CMA_CODE
0,5370001.08,2021S05075370001.08,0001.08,1.6383,35,"POLYGON ((-79.85362 43.19320, -79.85380 43.192...",537
1,0010002.00,2021S05070010002.00,0002.00,1.9638,10,"POLYGON ((-52.72050 47.55154, -52.71877 47.550...",001
2,5370001.09,2021S05075370001.09,0001.09,1.9699,35,"POLYGON ((-79.85586 43.18791, -79.85592 43.187...",537
3,5370120.02,2021S05075370120.02,0120.02,76.9650,35,"POLYGON ((-79.94562 43.16920, -79.94638 43.167...",537
4,0010006.00,2021S05070010006.00,0006.00,1.0467,10,"POLYGON ((-52.71107 47.56251, -52.71143 47.562...",001
...,...,...,...,...,...,...,...
6242,5591003.00,2021S05075591003.00,1003.00,227.6981,35,"POLYGON ((-82.63729 42.14866, -82.63820 42.136...",559
6243,5591004.00,2021S05075591004.00,1004.00,18.3792,35,"POLYGON ((-82.69416 42.05046, -82.69498 42.035...",559
6244,5800300.00,2021S05075800300.00,0300.00,314.4614,35,"POLYGON ((-80.41584 46.44983, -80.41636 46.444...",580
6245,6020800.00,2021S05076020800.00,0800.00,8.6987,46,"POLYGON ((-97.02578 49.59204, -97.02580 49.591...",602


Next, we read in a table that contains pre-prepared info: a list of provinces that match to PRUIDs, and a list of CMAs that matches names of CMAs and CAs to DGUIDs.

In [21]:
province_list = pd.read_csv("./data/provinces.csv").astype(str).set_index("PRUID")
cma_list = pd.read_csv("./data/cmas.csv").astype(str).set_index("ID")

Let's take a peek at the tracts table.

In [22]:
tracts.head(5)

,CTUID,DGUID,CTNAME,LANDAREA,PRUID,geometry,CMA_CODE
0,5370001.08,2021S05075370001.08,0001.08,1.6383,35,"POLYGON ((-79.85362 43.19320, -79.85380 43.192...",537
1,0010002.00,2021S05070010002.00,0002.00,1.9638,10,"POLYGON ((-52.72050 47.55154, -52.71877 47.550...",001
2,5370001.09,2021S05075370001.09,0001.09,1.9699,35,"POLYGON ((-79.85586 43.18791, -79.85592 43.187...",537
3,5370120.02,2021S05075370120.02,0120.02,76.9650,35,"POLYGON ((-79.94562 43.16920, -79.94638 43.167...",537
4,0010006.00,2021S05070010006.00,0006.00,1.0467,10,"POLYGON ((-52.71107 47.56251, -52.71143 47.562...",001


Because CMAs and CAs are coded a bit differently, we'll need to use the DGUID to get the CA or CMA code differently. To get the CA code from CA DGUIDs, we need to target the last three numbers before the decimal place. So let's remove the decimal place now to make this easier later on.

In [23]:
tracts["DGUID"] = tracts["DGUID"].str.replace("\.{1}[0-9A-B]+", "")

tracts

,CTUID,DGUID,CTNAME,LANDAREA,PRUID,geometry,CMA_CODE
0,5370001.08,2021S05075370001,0001.08,1.6383,35,"POLYGON ((-79.85362 43.19320, -79.85380 43.192...",537
1,0010002.00,2021S05070010002,0002.00,1.9638,10,"POLYGON ((-52.72050 47.55154, -52.71877 47.550...",001
2,5370001.09,2021S05075370001,0001.09,1.9699,35,"POLYGON ((-79.85586 43.18791, -79.85592 43.187...",537
3,5370120.02,2021S05075370120,0120.02,76.9650,35,"POLYGON ((-79.94562 43.16920, -79.94638 43.167...",537
4,0010006.00,2021S05070010006,0006.00,1.0467,10,"POLYGON ((-52.71107 47.56251, -52.71143 47.562...",001
...,...,...,...,...,...,...,...
6242,5591003.00,2021S05075591003,1003.00,227.6981,35,"POLYGON ((-82.63729 42.14866, -82.63820 42.136...",559
6243,5591004.00,2021S05075591004,1004.00,18.3792,35,"POLYGON ((-82.69416 42.05046, -82.69498 42.035...",559
6244,5800300.00,2021S05075800300,0300.00,314.4614,35,"POLYGON ((-80.41584 46.44983, -80.41636 46.444...",580
6245,6020800.00,2021S05076020800,0800.00,8.6987,46,"POLYGON ((-97.02578 49.59204, -97.02580 49.591...",602


First, we want to simplify our polygons a bit. Datawrapper has an upload size limit of 2MB, so we use `.simplify()` to reduce the size to an acceptable level. Let's first define a list of CMA code that are very large.

In [24]:
big_cities = [
    "535",
    "462",
    "559",
    "933",
    "205",
    "521",
    "421",
    "505",
    "835",
    "537"
]

Now, we simplify.

In [25]:
simple_tracts = tracts.copy()

simple_tracts.loc[simple_tracts["CMA_CODE"].isin(big_cities), "geometry"] = tracts["geometry"].simplify(tolerance=0.0001)

Now, we iterate through every CMA and CA in our list, and match that DGUID to the one in our shapefiles. Then, if there's data for that CMA, we output the file as a GeoJSON.

In [26]:
for id in cma_list.index.unique():

    name = cma_list.at[id, "NAME"].strip().lower().replace(" ", "")
    id_trim = id[-3:]
    type = cma_list.at[id, "TYPE"]
    prov_name = cma_list.at[id, "PROVINCE"]

    if type == "CMA":
        data = (simple_tracts
                .loc[simple_tracts["DGUID"].str.contains("2021S0507" + id_trim), :]
                )
    else:
        data = (simple_tracts
                .loc[simple_tracts["DGUID"].str[-3:] == id_trim, :]
                )

    if len(data) > 0:
        data.to_file(f"./data/cities/{prov_name.lower()}-{type.lower()}-{name}.geojson", driver='GeoJSON')


That's all. Now the repo should be populated with a list of useable GeoJSON files for Datawrapper maps based on the most recent data.

## Alberta maps

We're also going to produce maps for provinces using subdivisions and dissemination areas.

In [33]:
for prov_id in province_list.index.unique():
    
    prov_name = province_list.at[prov_id, "NAME"]

    # Loop through and create the dissemenation area maps.
    da = diss_areas[diss_areas["PRUID"] == prov_id]
    da["geometry"] = da["geometry"].simplify(tolerance=0.01)
    da.to_file(f"./data/provinces/da-{prov_name}.geojson", driver='GeoJSON')
    
    # Loop through and create the census subdivision maps.
    csd = subdivisions[subdivisions["PRUID"] == prov_id]
    csd["geometry"] = csd["geometry"].simplify(tolerance=0.01)
    csd.to_file(f"./data/provinces/csd-{prov_name}.geojson", driver='GeoJSON')

\-30\-